## Getting the data

In [1]:
import pandas as pd
import os
import numpy as np
from scipy import optimize
import numba as nb



In [20]:
%load_ext autoreload
%autoreload 2
from dataframefunctions import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
filename = 'germany.csv'
raw = read_csv_to_dataframe(filename)



In [15]:
print(raw)

   Party       Date  # of Seats  % of Seats  # of Votes  % of Votes Year
0    AfD  22-Sep-13           0        0.00     2052372        4.70   13
1    AfD  24-Sep-17          94       13.26     5878115       12.64   17
2    CDU  16-Oct-94         244       36.31    16089960       34.16   94
3    CDU  27-Sep-98         198       29.60    14004908       28.40   98
4    CDU  22-Sep-02         190       31.51    14167561       29.52   02
..   ...        ...         ...         ...         ...         ...  ...
89   SPD  22-Sep-02         251       41.63    18488688       38.52   02
90   SPD  18-Sep-05         222       36.16    16194665       34.25   05
91   SPD  27-Sep-09         146       23.47     9990488       23.03   09
92   SPD  22-Sep-13         193       30.59    11247283       25.74   13
93   SPD  24-Sep-17         153       21.58     9539381       20.51   17

[94 rows x 7 columns]


In [21]:
ger_data = transform_and_sort_dataframe(raw)

print(ger_data.head())

    Year Party  # of Seats
2   1994   CDU         244
9   1994   CSU          50
23  1994   FDP          47
32  1994   G -          49
69  1994   PDS          30


c:\Users\Fachschaft RW\Documents\sebbl workplace\masterarbeit\dataframefunctions.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\Fachschaft RW\Documents\sebbl workplace\masterarbeit\dataframefunctions.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
c:\Users\Fachschaft RW\Documents\sebbl workplace\masterarbeit\dataframefunctions.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [22]:
coalition_combinations = coalition_combinatorics_generator(ger_data)

for key, value in coalition_combinations.items():
    print(f"{key}: {value}")


('1994', 'CDU'): 244
('1994', 'CSU'): 50
('1994', 'FDP'): 47
('1994', 'G -'): 49
('1994', 'PDS'): 30
('1994', 'SPD'): 252
('1994', 'CDU+CSU'): 294
('1994', 'CDU+FDP'): 291
('1994', 'CDU+G -'): 293
('1994', 'CDU+PDS'): 274
('1994', 'CDU+SPD'): 496
('1994', 'CSU+FDP'): 97
('1994', 'CSU+G -'): 99
('1994', 'CSU+PDS'): 80
('1994', 'CSU+SPD'): 302
('1994', 'FDP+G -'): 96
('1994', 'FDP+PDS'): 77
('1994', 'FDP+SPD'): 299
('1994', 'G -+PDS'): 79
('1994', 'G -+SPD'): 301
('1994', 'PDS+SPD'): 282
('1994', 'CDU+CSU+FDP'): 341
('1994', 'CDU+CSU+G -'): 343
('1994', 'CDU+CSU+PDS'): 324
('1994', 'CDU+CSU+SPD'): 546
('1994', 'CDU+FDP+G -'): 340
('1994', 'CDU+FDP+PDS'): 321
('1994', 'CDU+FDP+SPD'): 543
('1994', 'CDU+G -+PDS'): 323
('1994', 'CDU+G -+SPD'): 545
('1994', 'CDU+PDS+SPD'): 526
('1994', 'CSU+FDP+G -'): 146
('1994', 'CSU+FDP+PDS'): 127
('1994', 'CSU+FDP+SPD'): 349
('1994', 'CSU+G -+PDS'): 129
('1994', 'CSU+G -+SPD'): 351
('1994', 'CSU+PDS+SPD'): 332
('1994', 'FDP+G -+PDS'): 126
('1994', 'FDP+G 

In [23]:
len(coalition_combinations)

473